# Part 2 

### Use Python’s Ollama library to prompt a model to annotate/classify song lyrics with their associated genre. Use the data (genreLyrics.csv) that is attached below. ###

Implement:
1) Zero-shot prompting
2) Few-Shot prompting

In [9]:
!pip install ollama

In [26]:
import pandas as pd
import ollama


In [11]:
file_path = "genreLyrics.csv"

#Reading csv
df = pd.read_csv(file_path, sep="\t", encoding="utf-8")

print(df.head())
print(df.columns)  


   Unnamed: 0       genre                                             lyrics
0       76301        Rock  Hey, if you were right I'd chase away\nAll the...
1      293332        Rock  There's something about the way we fit\nThere'...
2       70683  Electronic  One drop in the ocean\nCould be that magic pot...
3      209590        Rock  I'm so tired of being here\nSuppressed by all ...
4      116010     Hip-Hop  Yeah, what, Vast Aire,; Shell Shock..\nIt's th...
Index(['Unnamed: 0', 'genre', 'lyrics'], dtype='object')


In [12]:
print(df.sample(5))  #inspecting the csv

     Unnamed: 0  genre                                             lyrics
454      231062   Rock  I won't bind my strings to you\nBut build my w...
66       127999    Pop  (Verse 1)\nIf I never saw you again\ncould I n...
398       98783   Rock  Start the motor let the engine spin\nGive it f...
162      187043  Metal  When something stains your pride\nAnd leaves y...
300      223139   Jazz  Help my sister, did you make a mistake?\nThere...


In [13]:
print(df.head(20)) #supervising the csv file structure

    Unnamed: 0       genre                                             lyrics
0        76301        Rock  Hey, if you were right I'd chase away\nAll the...
1       293332        Rock  There's something about the way we fit\nThere'...
2        70683  Electronic  One drop in the ocean\nCould be that magic pot...
3       209590        Rock  I'm so tired of being here\nSuppressed by all ...
4       116010     Hip-Hop  Yeah, what, Vast Aire,; Shell Shock..\nIt's th...
5       186881        Rock  He said he'd be here at seven\nThe clock just ...
6       294994        Rock  Wir waren mehr als Freunde\nwir warn wie Brder...
7       182846        Rock  bokura ha sonna ni mo ooku no koto nado\nnozon...
8       315968         Pop  Love is free love is love\nThe world united\nI...
9       242844     Hip-Hop  Yeah, it's time to get crunked up in this bitc...
10      148419        Rock  Outside is a light\nWhich says you've got to f...
11      343109     Hip-Hop  Yes sir is Bobby Ray, aka A. B.O.B. 

In [14]:
# Print all the unique genres
genres = df['genre'].unique().tolist()
print("\nGenres found:", genres)


Genres found: ['Rock', 'Electronic', 'Hip-Hop', 'Pop', 'Other', 'R&B', 'Country', 'Jazz', 'Metal', 'Folk', 'Indie']


### Zero Shot Prompting

In [44]:
df = pd.read_csv("genreLyrics.csv", delimiter=r'\s*\t\s*', engine='python', on_bad_lines="skip", encoding="utf-8")

df.columns = df.columns.str.strip()

#List of unique genres, filtering out None values
genres = [genre for genre in df['genre'].unique().tolist() if genre is not None]
print("\nGenres found:", genres)

#Select first 50 sample lyrics for the better running of the engine
sample_lyrics = df['lyrics'].dropna().tolist()[:50]

#Zero-Shot Prompt Function
def get_zs_prompt(lyrics):
    return f"Classify the genre of the following song lyrics:\n\"{lyrics}\"\nAvailable genres: {', '.join(genres)}.\nReply with only the genre name."

#Classify 50 lyrics using Zero-Shot prompting
responses = []
for i, sample_lyric in enumerate(sample_lyrics):
    zs_prompt = get_zs_prompt(sample_lyric)
    
    #Query Ollama (Zero-Shot)
    zs_response = ollama.chat(model="llama3", messages=[{"role": "user", "content": zs_prompt}])
    
    #Store results
    responses.append((sample_lyric, zs_response['message']['content']))

#Printing the  results for all 50 lyrics
for i, (lyric, genre) in enumerate(responses):
    print(f"\nLyrics Sample {i+1}:\n", lyric)
    print("\nZero-Shot Predicted Genre:", genre)



Genres found: ['Rock', 'Electronic', 'Hip-Hop', 'Pop', 'Other', 'R&B', 'Country', 'Jazz', 'Metal', 'Folk', 'Indie', '-Boy style']

Lyrics Sample 1:
 "Hey, if you were right I'd chase away

Zero-Shot Predicted Genre: Pop

Lyrics Sample 2:
 "There's something about the way we fit

Zero-Shot Predicted Genre: Pop

Lyrics Sample 3:
 "One drop in the ocean

Zero-Shot Predicted Genre: Indie

Lyrics Sample 4:
 "I'm so tired of being here

Zero-Shot Predicted Genre: Indie

Lyrics Sample 5:
 "Yeah, what, Vast Aire,; Shell Shock..

Zero-Shot Predicted Genre: Hip-Hop

Lyrics Sample 6:
 "He said he'd be here at seven

Zero-Shot Predicted Genre: Pop

Lyrics Sample 7:
 "Wir waren mehr als Freunde

Zero-Shot Predicted Genre: Pop

Lyrics Sample 8:
 "bokura ha sonna ni mo ooku no koto nado

Zero-Shot Predicted Genre: J-Pop

Lyrics Sample 9:
 "Love is free love is love

Zero-Shot Predicted Genre: Pop

Lyrics Sample 10:
 "Yeah, it's time to get crunked up in this bitch

Zero-Shot Predicted Genre: Hip-Hop

## Few-Shot prompting

In [45]:
import pandas as pd
import ollama

df = pd.read_csv("genreLyrics.csv", delimiter=r'\s*\t\s*', engine='python', on_bad_lines="skip", encoding="utf-8")

df.columns = df.columns.str.strip()

genres = [genre for genre in df['genre'].unique().tolist() if genre is not None]
print("\nGenres found:", genres)

#Selecting the first 50 sample lyrics
sample_lyrics = df['lyrics'].dropna().tolist()[:50]

#Few-Shot Prompt Function
def get_fs_prompt(lyrics):
    # Randomly sample 3 examples of lyrics and genres from the dataset for Few-Shot
    few_shot_examples = df.dropna(subset=['lyrics', 'genre']).sample(n=3, random_state=1)
    examples = ""
    for _, row in few_shot_examples.iterrows():
        examples += f'Lyrics: "{row["lyrics"]}"\nGenre: {row["genre"]}\n\n'
    
    #Make the prompt
    return f"You are an expert in music genre classification. Here are some examples:\n\n{examples}Now, classify the following lyrics:\n\"{lyrics}\"\nReply with only the genre name."

#Classify the 50 lyrics using Few-Shot prompting
responses = []
for i, sample_lyric in enumerate(sample_lyrics):
    fs_prompt = get_fs_prompt(sample_lyric)
    
    #Ollama (Few-Shot)
    fs_response = ollama.chat(model="llama3", messages=[{"role": "user", "content": fs_prompt}])
    
    #Save the results results
    responses.append((sample_lyric, fs_response['message']['content']))

#Print the results for all 50 lyrics
for i, (lyric, genre) in enumerate(responses):
    print(f"\nLyrics Sample {i+1}:\n", lyric)
    print("\nFew-Shot Predicted Genre:", genre)



Genres found: ['Rock', 'Electronic', 'Hip-Hop', 'Pop', 'Other', 'R&B', 'Country', 'Jazz', 'Metal', 'Folk', 'Indie', '-Boy style']

Lyrics Sample 1:
 "Hey, if you were right I'd chase away

Few-Shot Predicted Genre: Rock

Lyrics Sample 2:
 "There's something about the way we fit

Few-Shot Predicted Genre: Pop

Lyrics Sample 3:
 "One drop in the ocean

Few-Shot Predicted Genre: Rock

Lyrics Sample 4:
 "I'm so tired of being here

Few-Shot Predicted Genre: Rock

Lyrics Sample 5:
 "Yeah, what, Vast Aire,; Shell Shock..

Few-Shot Predicted Genre: Hip-Hop/Rap

Lyrics Sample 6:
 "He said he'd be here at seven

Few-Shot Predicted Genre: Rock

Lyrics Sample 7:
 "Wir waren mehr als Freunde

Few-Shot Predicted Genre: Rock

Lyrics Sample 8:
 "bokura ha sonna ni mo ooku no koto nado

Few-Shot Predicted Genre: Rock

Lyrics Sample 9:
 "Love is free love is love

Few-Shot Predicted Genre: Pop

Lyrics Sample 10:
 "Yeah, it's time to get crunked up in this bitch

Few-Shot Predicted Genre: Rap/Hip-Hop



### Calculate the performance of both strategies of the model using Precision, Recall and F1 Score. Does performance vary by genre? ###

In [37]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [43]:
from sklearn.metrics import classification_report
import pandas as pd

true_labels = ['Rock', 'Rock', 'Electronic', 'Rock', 'Hip-Hop', 'Rock', 'Pop', 'Hip-Hop', 'Rock', 'Hip-Hop', 'Electronic', 'Rock', 'Pop', 'Electronic', 'Pop']  

#Zero-Shot Predictions
zs_predictions = ['Pop', 'Pop', 'Pop', 'Indie', 'Hip-Hop', 'Indie', 'Pop', 'J-Pop', 'Pop', 'Hip-Hop', 'Pop', 'Hip-Hop', 'Indie', 'Other', 'Rock']  

#Few-Shot Predictions
fs_predictions = ['Rock', 'Pop', 'Rock', 'Rock', 'Hip-Hop', 'Rock', 'Rock', 'J-Pop', 'Pop', 'Hip-Hop', 'Rock', 'Hip-Hop', 'Rock', 'Rock', 'Rock']  

#The unique genres
genres = ['Rock', 'Pop', 'Hip-Hop', 'Electronic', 'R&B', 'Country', 'Jazz', 'Metal', 'Indie', 'Folk']

#Clean the data
zs_clean = [p if p in genres else 'unknown' for p in zs_predictions]
fs_clean = [p if p in genres else 'unknown' for p in fs_predictions]

#Classification table for zero shot and few shot
print("\nFirst 15 results for Zero-Shot:")
print(pd.DataFrame({'Prediction': zs_clean, 'True Label': true_labels}).head(15))

print("\nFirst 15 results for Few-Shot:")
print(pd.DataFrame({'Prediction': fs_clean, 'True Label': true_labels}).head(15))

#Full classification table
print("\nZero-Shot Classification Report")
print(classification_report(true_labels, zs_clean, zero_division=0))

print('*' * 60)

print("\nFew-Shot Classification Report")
print(classification_report(true_labels, fs_clean, zero_division=0))



First 15 results for Zero-Shot:
   Prediction  True Label
0         Pop        Rock
1         Pop        Rock
2         Pop  Electronic
3       Indie        Rock
4     Hip-Hop     Hip-Hop
5       Indie        Rock
6         Pop         Pop
7     unknown     Hip-Hop
8         Pop        Rock
9     Hip-Hop     Hip-Hop
10        Pop  Electronic
11    Hip-Hop        Rock
12      Indie         Pop
13    unknown  Electronic
14       Rock         Pop

First 15 results for Few-Shot:
   Prediction  True Label
0        Rock        Rock
1         Pop        Rock
2        Rock  Electronic
3        Rock        Rock
4     Hip-Hop     Hip-Hop
5        Rock        Rock
6        Rock         Pop
7     unknown     Hip-Hop
8         Pop        Rock
9     Hip-Hop     Hip-Hop
10       Rock  Electronic
11    Hip-Hop        Rock
12       Rock         Pop
13       Rock  Electronic
14       Rock         Pop

Zero-Shot Classification Report
              precision    recall  f1-score   support

  Electronic   